In [ ]:
def CreateCorrMatrix(workingDir):
    os.chdir(workingDir)
    #===========================================================================
    #Import Receipts Data
    #===========================================================================
    receiptsRAW_df=pd.read_csv('201904 sales receipts.csv')

    #===========================================================================
    #Columns I want to use for this analyses
    #===========================================================================
    cols_to_keep=['transaction_date','sales_outlet_id','product_id','quantity','line_item_amount']
    receipts_df = receiptsRAW_df[receiptsRAW_df.columns.intersection(cols_to_keep)]

    #===========================================================================
    #Create month column
    #===========================================================================
    receipts_df['Month']=(pd.to_datetime(receipts_df['transaction_date']).dt.month).astype(int)

    #===========================================================================
    #Creating columns for April and May sales - These will eventually be 
    #aggregated
    #===========================================================================
    receipts_df['AprSales']=np.where(receipts_df['Month'] == 4,receipts_df['line_item_amount'], 0)

    receipts_df['MaySales']=np.where(receipts_df['Month'] == 5,receipts_df['line_item_amount'], 0)

    receipts_df['AprQtySales']=np.where(receipts_df['Month'] == 4,receipts_df['quantity'], 0)

    receipts_df['MayQtySales']=np.where(receipts_df['Month'] == 5,receipts_df['quantity'], 0)

    receipts_df['Store3_QtySales']=np.where(receipts_df['sales_outlet_id'] == 3,receipts_df['quantity'], 0)

    receipts_df['Store5_QtySales']=np.where(receipts_df['sales_outlet_id'] == 5,receipts_df['quantity'], 0)

    receipts_df['Store8_QtySales']=np.where(receipts_df['sales_outlet_id'] == 8,receipts_df['quantity'], 0)

    #===========================================================================
    #Drop unnecessary columns before grouping
    #===========================================================================
    receipts_df=receipts_df.drop(['transaction_date','sales_outlet_id','Month'], axis=1)

    #===========================================================================
    #Group by product ID to find total sales (by amount and quantity), monthly 
    #sales, and store sales
    #===========================================================================
    receipts_dfGrped=receipts_df.groupby(by=["product_id"],as_index=False).sum()

    #===========================================================================
    #Import products file
    #===========================================================================
    ProductsRAW_df=pd.read_csv('product.csv')

    #===========================================================================
    #Keep only essential columns for the following correlation analysis
    #===========================================================================
    cols_to_keep=['product_id','current_wholesale_price','current_retail_price']
    Products_df = ProductsRAW_df[ProductsRAW_df.columns.intersection(cols_to_keep)]

    #===========================================================================
    #Merge the receipts_dfGrped dataframe with the products_df dataframe to 
    #create an analytical dataset
    #===========================================================================
    Products_CorrADS=Products_df.merge(receipts_dfGrped,left_on='product_id',right_on='product_id',how='left')

    #===========================================================================
    #Coorelation analysis results
    #===========================================================================
    CoorResults=Products_CorrADS.corr(method="pearson")

    #===========================================================================
    #We're only interested in retail price's impact
    #===========================================================================
    testDF=pd.DataFrame(CoorResults['current_retail_price'].tail(7))

    #===========================================================================
    #Visualize the coorelation coefficients 
    #===========================================================================
    sns.heatmap(testDF, annot=True, vmin=-1, vmax=1, center=0,cmap=sns.diverging_palette(10, 133, as_cmap=True),square=True)
    plt.savefig('Corr_HeatMap.png')
    #===========================================================================
    #This code is just in case we want to visualize the rest of the matrix
    #===========================================================================
    #matrix = np.triu(CoorResults)
    #sns.heatmap(CoorResults, annot=True, mask=matrix)
    return Products_CorrADS

In [ ]:

def ProdCategorySales_Scatter(workingDir,Products_CorrADS):
    os.chdir(workingDir)
    #===========================================================================
    #Graphing Section
    #===========================================================================
    #For the graph I will need a table with:
        #product ID, 
        #product category, 
        #retail_price,
        #April sales

    #===========================================================================
    #Step 1: Import Product_id, retail_price, and April Sales
    #===========================================================================
    cols_to_keep=['product_id','current_retail_price','AprSales']
    Products_Plt_Step1 = Products_CorrADS[Products_CorrADS.columns.intersection(cols_to_keep)]

    #===========================================================================
    #Step 2: Import Product Category
    #===========================================================================
    cols_to_keep=['product_id','product_category']
    ProductsRAW_df=pd.read_csv('product.csv')
    ProductsCategory_df= ProductsRAW_df[ProductsRAW_df.columns.intersection(cols_to_keep)]

    #===========================================================================
    #Step 3: Merge - The result is the needed table
    #===========================================================================
    Products_PlotADS=Products_Plt_Step1.merge(ProductsCategory_df,left_on='product_id',right_on='product_id')

    #===========================================================================
    #Store the unique list of product category in a list - to be used later to 
    #loop through
    #===========================================================================
    UniqueProd_Category=list(dict.fromkeys(Products_PlotADS['product_category']))

    i=0
    #===========================================================================
    #Create an empty "plot" variable
    #===========================================================================
    fig, ax = plt.subplots()

    #===========================================================================
    #For each product category...
    #===========================================================================
    while i<=(len(UniqueProd_Category)-1):
        #===========================================================================
        #Store the product category as a string in a variable...
        #===========================================================================
        prodCat_toPlt=UniqueProd_Category[i]
        #===========================================================================
        #Filter the data for that product category
        #===========================================================================
        plot_df=Products_PlotADS[Products_PlotADS['product_category'] == prodCat_toPlt]
        #===========================================================================
        #Create a scatter plot with that filtered data
        #Note: The next iteration, the next "scatter plot", will just get added to 
        #      the previous iteration's, plot. 
        #===========================================================================
        ax.plot(plot_df['current_retail_price'],plot_df['AprSales'], 'o',label=prodCat_toPlt)
        i=i+1

    #===========================================================================
    #Format the Graph
    #===========================================================================
    ax.legend(bbox_to_anchor=(1,0.8))
    ax.set_ylabel('Sales') 
    ax.set_xlabel('Retail Price')
    ax.set_title('Sales v Retail Price per Product Category') 

    #===========================================================================
    #Format the axes to show as dollar values
    #===========================================================================
    ax.xaxis.set_major_formatter('${x:1.2f}')
    ax.yaxis.set_major_formatter('${x:1.2f}')

    plt.savefig('C:/Users/adit2/Documents/Syracuse/Scripting for Data Analysis/Final Project/Sales_V_PriceScatter.png')

    plt.show()

    #For heatmap & heatmap formatting
    #https://towardsdatascience.com/better-heatmaps-and-correlation-matrix-plots-in-python-41445d0f2bec

    #For Scatter plot formatting
    #https://matplotlib.org/3.3.2/gallery/pyplots/dollar_ticks.html#sphx-glr-gallery-pyplots-dollar-ticks-py

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

pd.options.mode.chained_assignment = None  # default='warn'
def runEverything(workingDir):
    Products_CorrADS=CreateCorrMatrix(workingDir)
    ProdCategorySales_Scatter(workingDir,Products_CorrADS)

print("Please input the directory where the RAW files are stored: ")
directory_str=input()
print("Directory: "+ directory_str)
directory_str=directory_str+"\\"
runEverything(directory_str)